In [ ]:
import torch
TORCH = torch.__version__.split('+')[0]
CUDA = 'cu' + torch.version.cuda.replace('.','')
print(TORCH, CUDA)

import pickle
import os

2.5.1 cu121


In [ ]:
%%capture
!pip install pyg_lib -f https://data.pyg.org/whl/torch-2.5.0+cu121.html
!pip install torch-scatter     -f https://data.pyg.org/whl/torch-2.5.0+cu121.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-2.5.0+cu121.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-2.5.0+cu121.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.5.0+cu121.html
!pip install torch-geometric

In [ ]:
import torch_geometric
import torch_geometric.nn as geom_nn
import torch_geometric.data as geom_data
from torch_geometric.loader import DataLoader
from torch_geometric.data import InMemoryDataset
from torch_geometric.data import Data

import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

from scipy.linalg import fractional_matrix_power
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform

import matplotlib.pyplot as plt
import matplotlib as matplotlib
import matplotlib.cm as cm

from tqdm import tqdm

import math
from numba import cuda
import numpy as np
import torch
from torch_geometric.utils import (
    to_networkx,
    from_networkx,
    to_dense_adj,
    remove_self_loops,
    to_undirected,
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Set up Functions

In [ ]:
def data_to_kNN(X,k):
  edge_index = torch_geometric.nn.knn_graph(X, k)
  edge_index = to_undirected(edge_index, num_nodes=X.shape[0])
  return edge_index

def data_to_GNN_data(X,Y,k):
  dataset = []
  N = X.shape[0]
  for i in range(N):
    data = Data(x = X[i,:,:], y = Y[i,:,:], edge_index = data_to_kNN(X[i,:,:],k))
    data.num_nodes = 20
    data.num_edges = data.edge_index.shape[1]
    dataset.append(data)
  return dataset

def data_to_GNN_data_fm(X,Y,k):
  dataset = []
  N = len(X)
  for i in tqdm(range(N)):
    data = Data(x = X[i], y = Y[i], edge_index = data_to_kNN(X[i],k))
    data.num_nodes = X[i].shape[0]
    data.num_edges = data.edge_index.shape[1]
    dataset.append(data)
  return dataset

In [ ]:
# Cucker-Smale Consensus Model (2D)

import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt
import torch

def CS(t,z,N):

  dxdt = np.zeros(N); dydt = np.zeros(N)
  dudt = np.zeros(N); dvdt = np.zeros(N)

  x = z[0:N]; y = z[N:2*N]
  u = z[2*N:3*N]; v = z[3*N:]

  Fx = np.zeros((N,N)); Fy = np.zeros((N,N))

  for i in range(0,N):
    for j in range(0,N):
        Fx[i,j] = (1/N)*Phi(np.linalg.norm(np.array([x[i]-x[j], y[i]-y[j]])))*(u[j]-u[i])
        Fy[i,j] = (1/N)*Phi(np.linalg.norm(np.array([x[i]-x[j], y[i]-y[j]])))*(v[j]-v[i])

  for i in range(0,N):
    dxdt[i] = u[i]
    dydt[i] = v[i]
    dudt[i] = np.sum(Fx[i,:])
    dvdt[i] = np.sum(Fy[i,:])

  dzdt = np.concatenate((dxdt, dydt, dudt, dvdt))
  return dzdt

def Phi(r):
   H = 1; s = 1; b = 1
   y = H/(s**2+r**2)**b
   return y

# %%%%%%%%%%%%%%%%%%%%%%% Main %%%%%%%%%%%%%%%%%%%%%%%

N = 20 # Number of agents
T = 100 # Final time

x0 = np.random.uniform(0,1,N)
y0 = np.random.uniform(0,1,N)
u0 = np.random.uniform(0,1,N)
v0 = np.random.uniform(0,1,N)
z0 = np.concatenate((x0,y0,u0,v0))

t_eval = torch.tensor(range(0,500))*T/500

soln = integrate.solve_ivp(CS, (0, T), z0, method='BDF',args=[N])

x = soln.y[0:N,:]; y = soln.y[N:2*N,:];
u = soln.y[2*N:3*N,:]; v = soln.y[3*N:4*N,:]

In [ ]:
import torch
from tqdm import tqdm

X = torch.tensor(soln.y)
t = torch.tensor(soln.t)

N = 20 # Number of agents
T_end = 100 # Final time
N_initials = 500 # Number of initializations


def create_data_one_initialization(N,T_end):
    x0 = np.random.uniform(0,1,N)
    y0 = np.random.uniform(0,1,N)
    u0 = np.random.uniform(0,1,N)
    v0 = np.random.uniform(0,1,N)
    z0 = np.concatenate((x0,y0,u0,v0))

    soln = integrate.solve_ivp(CS, (0, T_end), z0, method='BDF',args=[N])

    X = torch.tensor(soln.y) # 4N x t =[x;y;xdot;ydot] X t
    t = torch.tensor(soln.t)

    Xtrn, Ytrn = create_y(X,t)
    Xtrn = resize_tensor(Xtrn)
    Ytrn = resize_tensor(Ytrn)

    return Xtrn, Ytrn

def create_data(N, T_end, N_initials):
  Xtrn, Ytrn = create_data_one_initialization(N,T_end)
  for i in tqdm(range(N_initials-1)):
    Xtrn_new, Ytrn_new = create_data_one_initialization(N,T_end)

    Xtrn = torch.cat((Xtrn, Xtrn_new), dim = 0)
    Ytrn = torch.cat((Ytrn, Ytrn_new), dim = 0)

  return Xtrn, Ytrn

def create_y(X,t):
  N = X.shape[0]
  T = X.shape[1]

  n = N//4

  y = torch.zeros(N,T)
  for i in range(1,T-1):
    y[:n,i] = X[2*n:3*n, i]
    y[n:2*n,i] = X[3*n:, i]
    y[2*n:,i] = (X[2*n:,i+1] - X[2*n:,i-1])/(t[i+1]-t[i-1])

  return X[:,1:-1], y[:,1:-1]

def resize_tensor(A):
  N, T = A.shape
  n = N//4
  new_A = torch.zeros(T, N//4, 4)

  new_A[:,:,0] = A.T[:,:n]
  new_A[:,:,1] = A.T[:,n:2*n]
  new_A[:,:,2] = A.T[:,2*n:3*n]
  new_A[:,:,3] = A.T[:,3*n:]

  return new_A

# Set up the Models

In [ ]:
import torch
import scipy.io as sio
import torch.nn as nn

In [ ]:
# Throughout this we are going to assume that data is of the form B x N x D
# Where B is the batch size, N is the sequence length for the transformer
# this the number of data points. Finally D is the embedding dimension.
class SimpleAttention(nn.Module):
  # Initialize the parameter
  def __init__(self, hidden_dim):
    super(SimpleAttention, self).__init__()
    self.linear = nn.Linear(hidden_dim, hidden_dim)
    self.WQ = nn.Linear(hidden_dim, hidden_dim, bias = False)
    self.WK = nn.Linear(hidden_dim, hidden_dim, bias = False)
    self.WV = nn.Linear(hidden_dim, hidden_dim, bias = False)
    self.skip = nn.Linear(hidden_dim, hidden_dim)
    self.attention = nn.MultiheadAttention(hidden_dim, 1, batch_first=True)

  # Forward pass
  def forward(self, input):
    Q = self.WQ(input)
    K = self.WK(input)
    V = self.WV(input)
    output_attention,_ = self.attention(Q,K,V)
    output_linear = self.linear(output_attention.relu()).relu()
    return output_linear + self.skip(input)

class SimpleTransformer(nn.Module):
# Initialize the parameter
  def __init__(self, input_dim, hidden_dim, out_dim, num_layers):
    super(SimpleTransformer, self).__init__()
    self.embed = nn.Linear(input_dim, hidden_dim)
    self.predictor = nn.Linear(hidden_dim, out_dim)
    self.AttentionLayers = []
    for i in range(num_layers):
      self.AttentionLayers.append(SimpleAttention(hidden_dim))

    self.AttentionLayers = nn.ModuleList(self.AttentionLayers)
    self.num_layers = num_layers

  # Forward pass
  def forward(self, z):
    z = self.embed(z)
    for i in range(self.num_layers):
      z = self.AttentionLayers[i](z)
    return self.predictor(z)


class FNN(nn.Module):
  def __init__(self, input_dim, hidden_dim, out_dim, num_layers, d):
    super(FNN, self).__init__()
    self.embed = nn.Linear(input_dim, hidden_dim)
    self.predictor = nn.Linear(hidden_dim, out_dim)
    self.Layers = []
    for i in range(num_layers):
      self.Layers.append(nn.Linear(hidden_dim, hidden_dim))

    self.Layers = nn.ModuleList(self.Layers)
    self.num_layers = num_layers
    self.d = d

  # Input size B x N X 4
  def forward(self,x):
    x = torch.flatten(x, 1, 2) # B x N4
    x = self.embed(x).relu()
    for i in range(self.num_layers):
      x = self.Layers[i](x).relu()
    x = self.predictor(x)
    x = torch.unflatten(x, 1, (-1,self.d))
    return x

def kernel_basis(X, d1, d2):
  N = X.shape[0]
  d = X.shape[1]

  Phi = torch.zeros(N, (d1+2*d2)*d, device = X.device)
  for i in range(d1+2*d2):
    if i < d1:
      for j in range(d):
        Phi[:,d*i+j] = X[:,j].pow(i+1)
    elif i < d1+d2:
      for j in range(d):
        k = i-d1+1
        Phi[:,d*i+j] = (X[:,j]*k).sin()
    else:
      for j in range(d):
        k = i-d1-d2+1
        Phi[:,d*i+j] = torch.cos(X[:,j]*k)

  return Phi

class Kernel(nn.Module):
  def __init__(self, feature_dim, out_dim, d, embed=kernel_basis):
    super(Kernel, self).__init__()
    self.predictor = nn.Linear(feature_dim, out_dim, device = "cuda")
    self.embed = embed
    self.d = d

  # Input size B x N X 4
  def forward(self,x):
    x = self.predictor(x)
    x = torch.unflatten(x, 1, (-1,self.d))
    return x


In [ ]:
def get_nn(in_channels, out_channels):
  return torch.nn.Sequential(torch.nn.Linear(in_channels, out_channels), torch.nn.ReLU(),
                             torch.nn.Linear(out_channels, out_channels))

class GNN(torch.nn.Module):
  def __init__(self, node_input_dim, output_dim, num_layers, hidden_dim = 128, device = "cuda", arch = "Graph"):
    super().__init__()

    self.num_layers = num_layers
    self.layers = []
    if arch == "Transformer":
      self.layers.append(torch_geometric.nn.TransformerConv(node_input_dim, hidden_dim).to(device))
      for i in range(num_layers-1):
        self.layers.append(torch_geometric.nn.TransformerConv(hidden_dim, hidden_dim).to(device))
    elif arch == "Graph":
      self.layers.append(torch_geometric.nn.GraphConv(node_input_dim, hidden_dim).to(device))
      for i in range(num_layers-1):
        self.layers.append(torch_geometric.nn.GraphConv(hidden_dim, hidden_dim).to(device))
    elif arch == "GIN":
      self.layers.append(torch_geometric.nn.GINConv(get_nn(node_input_dim, hidden_dim)).to(device))
      for i in range(num_layers-1):
        self.layers.append(torch_geometric.nn.GINConv(get_nn(hidden_dim, hidden_dim)).to(device))

    self.layers = torch.nn.ModuleList(self.layers)
    self.lin = torch.nn.Linear(hidden_dim, output_dim).to(device)

  def forward(self, x, edge_index, batch):
    for i in range(self.num_layers):
      x = self.layers[i](x, edge_index).relu()

    z = self.lin(x)
    return z

In [ ]:
class Cylindrical_FNN(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, out_dim, num_layers):
    super(Cylindrical_FNN, self).__init__()
    self.layers = []
    self.layers.append(nn.Linear(input_dim, hidden_dim))
    for i in range(num_layers-2):
      self.layers.append(nn.Linear(hidden_dim, hidden_dim))
    self.layers.append(nn.Linear(hidden_dim, out_dim))

    self.layers = nn.ModuleList(self.layers)

  def forward(self, x):
    for i in range(len(self.layers)):
      x = self.layers[i](x).relu()
    return x

class Cylindrical(torch.nn.Module):
  def __init__(self, node_input_dim, output_dim, num_layers_psi, num_layers_phi, hidden_dim = 128, device = "cuda"):
    super().__init__()

    self.psi = Cylindrical_FNN(node_input_dim, hidden_dim, hidden_dim, num_layers_psi).to(device)
    self.phi = Cylindrical_FNN(hidden_dim+node_input_dim, hidden_dim, output_dim, num_layers_phi).to(device)

  def forward(self, x):
    # print(x.shape)
    z = self.psi(x)
    # print(z.shape, z.mean(dim=1).shape)
    z_mean = z.mean(dim=1).unsqueeze(1).repeat(1, x.shape[1], 1)
    # print(z_mean.shape)
    z = torch.cat((z_mean,x), dim=-1)
    z = self.phi(z)

    return z

# Training the Model

In [ ]:
Xtrn, Ytrn = torch.load("drive/MyDrive/MeasureMaps/data-cucker-2.pt")

train_idx = torch.load("drive/MyDrive/MeasureMaps/Cucker-data-split.pt")[0]

# train_X = [Xtrn[i] for i in train_idx]
# train_Y = [Ytrn[i] for i in train_idx]

# dataset_knn = data_to_GNN_data(Xtrn, Ytrn, 3)
# dataset_full = data_to_GNN_data(Xtrn, Ytrn, 20)
train_data = torch.utils.data.TensorDataset(Xtrn[train_idx,:,:], Ytrn[train_idx,:,:])

train_loader = torch.utils.data.DataLoader(train_data, shuffle = True, batch_size=500)
# train_loader_gnn_full = DataLoader([dataset_full[i] for i in train_idx.numpy()], shuffle = True, batch_size = 500)
# train_loader_gnn_knn = DataLoader([dataset_knn[i] for i in train_idx.numpy()], shuffle = True, batch_size = 500)

<ipython-input-15-627000409fd4>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  Xtrn, Ytrn = torch.load("drive/MyDrive/MeasureMaps/data-cucker-2.pt")
<ipython-input-15-6270

In [ ]:
from tqdm import tqdm

depths = [3,4,5]
widths = [128,256,512]
lrs = [2e-4, 1e-3, 1e-4]
T = 5

epochs = 1001

for t in range(T):
  for d in depths: # depth
    for h in widths: # width
      for lr in lrs:
        if os.path.exists("drive/MyDrive/MeasureMaps/Cylindrical/trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-"+str(10)+".pt"):
          print("Done with",t,d,h,lr)
          continue
        model_cylin = Cylindrical(4,4,d//2, d - d//2, h).to('cuda')

        optimizer_cylin = torch.optim.Adam(model_cylin.parameters(), lr = lr)
        scheduler_cylin = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_cylin, epochs)

        for i in tqdm(range(epochs)):
          for X,Y in train_loader:
            optimizer_cylin.zero_grad()
            X = X.to('cuda')
            Y = Y.to('cuda')
            Y_pred = model_cylin(X)
            loss = torch.nn.functional.mse_loss(Y_pred, Y)
            loss.backward()
            optimizer_cylin.step()
          if i % 100 == 0:
            torch.save(model_cylin,"drive/MyDrive/MeasureMaps/Cylindrical/trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-"+str(i)+".pt")
          scheduler_cylin.step()

100%|██████████| 1001/1001 [02:45<00:00,  6.06it/s]


In [ ]:
depths = [3,4,5]
widths = [128,256,512]
lrs = [1e-4, 2e-4, 1e-3]
T = 5

epochs = 1001

for t in range(3,T):
  for d in depths:
    for h in widths:
      for lr in lrs:
        model_transformer = SimpleTransformer(4,h,4,d).to('cuda')

        optimizer_transformer = torch.optim.Adam(model_transformer.parameters(), lr = lr)
        scheduler_transformer = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_transformer, epochs)

        for i in tqdm(range(epochs)):
          for X,Y in train_loader:
            optimizer_transformer.zero_grad()
            Y_pred = model_transformer(X)
            loss = torch.nn.functional.mse_loss(Y_pred, Y)
            loss.backward()
            optimizer_transformer.step()
          if i % 100 == 0:
            torch.save(model_transformer,"drive/MyDrive/MeasureMaps/Transformer/trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-"+str(i)+".pt")
          scheduler_transformer.step()

In [ ]:
degree = [2,3,4]
frequencies = [4,5,6]
lrs = [1e-4, 2e-4, 1e-3]
T = 5

epochs = 1001

for t in range(T):
  for d1 in degree:
    for d2 in frequencies:
      for lr in lrs:
        Xtrn_data_kernel = kernel_basis(torch.flatten(Xtrn_data,1,2),d1,d2)
        train_data_kernel = torch.utils.data.TensorDataset(Xtrn_data_kernel[train_idx,:], Ytrn_data[train_idx,:,:])
        train_loader_kernel = torch.utils.data.DataLoader(train_data_kernel, shuffle = True, batch_size=500)

        model_kernel = Kernel((d1+2*d2)*4*20,80,4).to('cuda')

        optimizer_kernel = torch.optim.Adam(model_kernel.parameters(), lr = lr)
        scheduler_kernel = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_kernel, epochs)

        for i in tqdm(range(epochs)):
          for X,Y in train_loader_kernel:
            optimizer_kernel.zero_grad()
            Y_pred = model_kernel(X)
            loss = torch.nn.functional.mse_loss(Y_pred, Y)
            loss.backward()
            optimizer_kernel.step()
          if i % 100 == 0:
            torch.save(model_kernel,"drive/MyDrive/MeasureMaps/Kernel/trial-"+str(t)+"-degree-"+str(d1)+"-frequency-"+str(d2)+"-lr-"+str(lr)+"-epoch-"+str(i)+".pt")
          scheduler_kernel.step()

In [ ]:
from tqdm import tqdm

depths = [3,4,5]
widths = [128,256,512]
lrs = [1e-4, 2e-4, 1e-3]
T = 5
N = 20

epochs = 1001

for t in range(1):
  for d in depths:
    for h in widths:
      for lr in lrs:
        model_fnn = FNN(4*N,h,4*N,d,4).to('cuda')


        optimizer_fnn = torch.optim.Adam(model_fnn.parameters(), lr = lr)
        scheduler_fnn = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_fnn, epochs)

        for i in tqdm(range(epochs)):
          for X,Y in train_loader:
            optimizer_fnn.zero_grad()
            Y_pred = model_fnn(X)
            loss = torch.nn.functional.mse_loss(Y_pred, Y)
            loss.backward()
            optimizer_fnn.step()
          if i % 100 == 0:
            torch.save(model_fnn,"drive/MyDrive/MeasureMaps/FNN/trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-"+str(i)+".pt")
          scheduler_fnn.step()

In [ ]:
from tqdm import tqdm

depths = [3,4,5]
widths = [128,256,512]
lrs = [1e-4, 2e-4, 1e-3]
T = 5

epochs = 1001

for t in range(T):
  for d in depths:
    for h in widths:
      for lr in lrs:
        model_gnn = GNN(4,4,d,h).to('cuda')


        optimizer_gnn = torch.optim.Adam(model_gnn.parameters(), lr = lr)
        scheduler_gnn = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_gnn, epochs)

        for i in tqdm(range(epochs)):
          for data in train_loader_gnn_full:
            optimizer_gnn.zero_grad()
            data = data.to('cuda')
            Y_pred = model_gnn(data.x, data.edge_index, data.batch)
            loss = torch.nn.functional.mse_loss(Y_pred, data.y)
            loss.backward()
            optimizer_gnn.step()
          if i % 100 == 0:
            torch.save(model_gnn,"drive/MyDrive/MeasureMaps/GNN/GraphConv Full/trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-"+str(i)+".pt")
          scheduler_gnn.step()

In [ ]:
depths = [3,4,5]
widths = [128,256,512]
lrs = [1e-4, 2e-4, 1e-3]
T = 5

epochs = 1001

for t in range(T):
  for d in depths:
    for h in widths:
      for lr in lrs:
        model_gnn = GNN(4,4,d,h).to('cuda')


        optimizer_gnn = torch.optim.Adam(model_gnn.parameters(), lr = lr)
        scheduler_gnn = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_gnn, epochs)

        for i in tqdm(range(epochs)):
          for data in train_loader_gnn_knn:
            optimizer_gnn.zero_grad()
            data = data.to('cuda')
            Y_pred = model_gnn(data.x, data.edge_index, data.batch)
            loss = torch.nn.functional.mse_loss(Y_pred, data.y)
            loss.backward()
            optimizer_gnn.step()
          if i % 100 == 0:
            torch.save(model_gnn,"drive/MyDrive/MeasureMaps/GNN/GraphConv KNN/trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-"+str(i)+".pt")
          scheduler_gnn.step()

In [ ]:
depths = [3,4,5]
widths = [128,256,512]
lrs = [1e-4, 2e-4, 1e-3]
T = 5

epochs = 1001

for t in range(T):
  for d in depths:
    for h in widths:
      for lr in lrs:
        model_gnn = GNN(4,4,d,h,arch="Transformer").to('cuda')


        optimizer_gnn = torch.optim.Adam(model_gnn.parameters(), lr = lr)
        scheduler_gnn = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_gnn, epochs)

        for i in tqdm(range(epochs)):
          for data in train_loader_gnn_full:
            optimizer_gnn.zero_grad()
            data = data.to('cuda')
            Y_pred = model_gnn(data.x, data.edge_index, data.batch)
            loss = torch.nn.functional.mse_loss(Y_pred, data.y)
            loss.backward()
            optimizer_gnn.step()
          if i % 100 == 0:
            torch.save(model_gnn,"drive/MyDrive/MeasureMaps/GNN/TransformerConv Full/trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-"+str(i)+".pt")
          scheduler_gnn.step()

In [ ]:
depths = [3,4,5]
widths = [128,256,512]
lrs = [1e-4, 2e-4, 1e-3]
T = 5

epochs = 1001

for t in range(T):
  for d in depths:
    for h in widths:
      for lr in lrs:
        model_gnn = GNN(4,4,d,h,arch="Transformer").to('cuda')


        optimizer_gnn = torch.optim.Adam(model_gnn.parameters(), lr = lr)
        scheduler_gnn = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_gnn, epochs)

        for i in tqdm(range(epochs)):
          for data in train_loader_gnn_knn:
            optimizer_gnn.zero_grad()
            data = data.to('cuda')
            Y_pred = model_gnn(data.x, data.edge_index, data.batch)
            loss = torch.nn.functional.mse_loss(Y_pred, data.y)
            loss.backward()
            optimizer_gnn.step()
          if i % 100 == 0:
            torch.save(model_gnn,"drive/MyDrive/MeasureMaps/GNN/TransformerConv KNN/trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-"+str(i)+".pt")
          scheduler_gnn.step()

# Testing Models

In [ ]:
Xtrn, Ytrn = torch.load("drive/MyDrive/MeasureMaps/data-cucker-2.pt")
idxes = torch.load("drive/MyDrive/MeasureMaps/Cucker-data-split.pt")

<ipython-input-29-b9586dbbda54>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  Xtrn, Ytrn = torch.load("drive/MyDrive/MeasureMaps/data-cucker-2.pt")
<ipython-input-29-b958

In [ ]:
N = Xtrn.shape[0]
val_idx = idxes[1]
test_idx = idxes[2]
print(val_idx.shape, test_idx.shape)

torch.Size([1684]) torch.Size([1683])


In [ ]:
val_data = torch.utils.data.TensorDataset(Xtrn[val_idx,:], Ytrn[val_idx,:,:])
val_loader = torch.utils.data.DataLoader(val_data, shuffle = True, batch_size=421)

test_data = torch.utils.data.TensorDataset(Xtrn[test_idx,:], Ytrn[test_idx,:,:])
test_loader = torch.utils.data.DataLoader(test_data, shuffle = True, batch_size=421)

In [ ]:
def test_transformer_fnn_cs(model, loader):
  N = 4
  loss = torch.zeros(N)
  model.eval()
  i = 0
  for X,Y in loader:
    X = X.to('cuda')
    Y = Y.to('cuda')
    Y_pred = model(X)
    loss[i] = torch.nn.functional.mse_loss(Y_pred, Y).cpu().detach()
  return loss

In [ ]:
dataset_knn = data_to_GNN_data(Xtrn.to('cuda'), Ytrn.to('cuda'), 3)
dataset_full = data_to_GNN_data(Xtrn.to('cuda'), Ytrn.to('cuda'), 20)

In [ ]:
val_loader_gnn_full = DataLoader([dataset_full[i] for i in val_idx.numpy()], shuffle = True, batch_size = 1)
val_loader_gnn_knn = DataLoader([dataset_knn[i] for i in val_idx.numpy()], shuffle = True, batch_size = 1)

test_loader_gnn_full = DataLoader([dataset_full[i] for i in test_idx.numpy()], shuffle = True, batch_size = 1)
test_loader_gnn_knn = DataLoader([dataset_knn[i] for i in test_idx.numpy()], shuffle = True, batch_size = 1)

In [ ]:
depths = [3,4,5]
widths = [128,256,512]
lrs = [2e-4, 1e-3, 1e-4]
T = 5

best_loss = 100000
best_d = -1
best_w = -1
best_lr = -1

for d in depths: # depth
  for h in widths: # width
    for lr in lrs:
      avg_loss = 0
      print("Testing: ",d,h,lr)
      for t in range(T):
        model = torch.load("drive/MyDrive/MeasureMaps/FNN/trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-1000.pt")
        loss = test_transformer_fnn_cs(model, val_loader)
        avg_loss = loss.mean()
        print(avg_loss)
      if avg_loss < best_loss:
        best_loss = avg_loss
        best_d = d
        best_w = h
        best_lr = lr
print(best_loss, best_d, best_w, best_lr)

Testing:  3 128 0.0002


<ipython-input-37-96cea253544d>:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("drive/MyDrive/MeasureMaps/FNN/trial-"+str(t)+"-depth-"+str(d)+"-width-"

tensor(2.1363e-05)
tensor(2.1230e-05)
tensor(2.1030e-05)
tensor(2.0713e-05)
tensor(2.2510e-05)
Testing:  3 128 0.001
tensor(2.3446e-05)
tensor(1.9242e-05)
tensor(2.0489e-05)
tensor(2.1375e-05)
tensor(2.1616e-05)
Testing:  3 128 0.0001
tensor(2.1151e-05)
tensor(2.2673e-05)
tensor(2.0938e-05)
tensor(2.0582e-05)
tensor(2.4527e-05)
Testing:  3 256 0.0002
tensor(1.8158e-05)
tensor(1.8104e-05)
tensor(1.6645e-05)
tensor(1.7716e-05)
tensor(1.6924e-05)
Testing:  3 256 0.001
tensor(1.6937e-05)
tensor(1.6155e-05)
tensor(1.6524e-05)
tensor(1.5239e-05)
tensor(1.7330e-05)
Testing:  3 256 0.0001
tensor(1.7923e-05)
tensor(1.9017e-05)
tensor(1.7124e-05)
tensor(1.7812e-05)
tensor(1.7269e-05)
Testing:  3 512 0.0002
tensor(1.3897e-05)
tensor(1.4825e-05)
tensor(1.5301e-05)
tensor(1.2498e-05)
tensor(1.5064e-05)
Testing:  3 512 0.001
tensor(7.9684e-06)
tensor(8.9309e-06)
tensor(9.9711e-06)
tensor(8.9639e-06)
tensor(9.2344e-06)
Testing:  3 512 0.0001
tensor(1.5717e-05)
tensor(1.4820e-05)
tensor(1.4496e-05)
te

In [ ]:
loss_avg = torch.zeros(5)
loss_std = torch.zeros(5)
for t in range(T):
  model = torch.load("drive/MyDrive/MeasureMaps/FNN/trial-"+str(t)+"-depth-"+str(best_d)+"-width-"+str(best_w)+"-lr-"+str(best_lr)+"-epoch-1000.pt")
  loss = test_transformer_fnn_cs(model, test_loader)
  loss_avg[t] = loss.mean()
  loss_std[t] = loss.std()
print(loss_avg.mean(), loss_avg.std())

tensor(6.8460e-06) tensor(5.0391e-07)


<ipython-input-38-792fd64ffe90>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("drive/MyDrive/MeasureMaps/FNN/trial-"+str(t)+"-depth-"+str(best_d)+"-wid

In [ ]:
depths = [3,4,5]
widths = [128,256,512]
lrs = [2e-4, 1e-3, 1e-4]
T = 5

best_loss = 100000
best_d = -1
best_w = -1
best_lr = -1

for d in depths: # depth
  for h in widths: # width
    for lr in lrs:
      avg_loss = 0
      print("Testing: ",d,h,lr)
      for t in range(T):
        model = torch.load("drive/MyDrive/MeasureMaps/Cylindrical/trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-1000.pt")
        loss = test_transformer_fnn_cs(model, val_loader)
        avg_loss = loss.mean()
        print(avg_loss)
      if avg_loss < best_loss:
        best_loss = avg_loss
        best_d = d
        best_w = h
        best_lr = lr
print(best_loss, best_d, best_w, best_lr)

Testing:  3 128 0.0002
tensor(0.0176)
tensor(0.0181)
tensor(0.0179)
tensor(0.0011)
tensor(0.0183)
Testing:  3 128 0.001
tensor(0.0011)
tensor(0.0013)
tensor(0.0011)
tensor(0.0010)


<ipython-input-41-9ccd5f4c0f66>:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("drive/MyDrive/MeasureMaps/Cylindrical/trial-"+str(t)+"-depth-"+str(d)+"

tensor(0.0012)
Testing:  3 128 0.0001
tensor(0.0013)
tensor(0.0181)
tensor(0.0011)
tensor(0.0010)
tensor(0.0008)
Testing:  3 256 0.0002
tensor(0.0006)
tensor(0.0008)
tensor(0.0008)
tensor(0.0017)
tensor(0.0016)
Testing:  3 256 0.001
tensor(0.0012)
tensor(0.0017)
tensor(0.0015)
tensor(0.0179)
tensor(0.0011)
Testing:  3 256 0.0001
tensor(0.0011)
tensor(0.0008)
tensor(0.0008)
tensor(0.0179)
tensor(0.0007)
Testing:  3 512 0.0002
tensor(0.0011)
tensor(0.0014)
tensor(0.0011)
tensor(0.0015)
tensor(0.0011)
Testing:  3 512 0.001
tensor(0.0015)
tensor(0.0011)
tensor(0.0014)
tensor(0.0015)
tensor(0.0016)
Testing:  3 512 0.0001
tensor(0.0011)
tensor(0.0015)
tensor(0.0014)
tensor(0.0016)
tensor(0.0015)
Testing:  4 128 0.0002
tensor(0.0011)
tensor(0.0015)
tensor(0.0010)
tensor(0.0007)
tensor(0.0011)
Testing:  4 128 0.001
tensor(0.0012)
tensor(0.0015)
tensor(0.0015)
tensor(0.0012)
tensor(0.0012)
Testing:  4 128 0.0001
tensor(0.0173)
tensor(0.0009)
tensor(0.0012)
tensor(0.0007)
tensor(0.0007)
Testing:

In [ ]:
loss_avg = torch.zeros(5)
loss_std = torch.zeros(5)
for t in range(T):
  model = torch.load("drive/MyDrive/MeasureMaps/Cylindrical/trial-"+str(t)+"-depth-"+str(best_d)+"-width-"+str(best_w)+"-lr-"+str(best_lr)+"-epoch-1000.pt")
  loss = test_transformer_fnn_cs(model, test_loader)
  loss_avg[t] = loss.mean()
  loss_std[t] = loss.std()
print(loss_avg.mean(), loss_avg.std())

tensor(0.0011) tensor(0.0004)


<ipython-input-42-ab4b33ff4b7a>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("drive/MyDrive/MeasureMaps/Cylindrical/trial-"+str(t)+"-depth-"+str(best_

In [ ]:
depths = [3,4,5]
widths = [128,256,512]
lrs = [2e-4, 1e-3, 1e-4]
T = 5

best_loss = 100000
best_d = -1
best_w = -1
best_lr = -1

for d in depths: # depth
  for h in widths: # width
    for lr in lrs:
      avg_loss = 0
      print("Testing: ",d,h,lr)
      for t in range(T):
        model = torch.load("drive/MyDrive/MeasureMaps/Transformer/trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-1000.pt")
        loss = test_transformer_fnn_cs(model, val_loader)
        avg_loss = loss.mean()
        print(avg_loss)
      if avg_loss < best_loss:
        best_loss = avg_loss
        best_d = d
        best_w = h
        best_lr = lr
print(best_loss, best_d, best_w, best_lr)

Testing:  3 128 0.0002


<ipython-input-39-8886248ee820>:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("drive/MyDrive/MeasureMaps/Transformer/trial-"+str(t)+"-depth-"+str(d)+"

tensor(5.2892e-06)
tensor(2.3748e-06)
tensor(4.6765e-06)
tensor(5.5105e-06)
tensor(5.7769e-06)
Testing:  3 128 0.001
tensor(6.5881e-06)
tensor(5.0327e-06)
tensor(5.3223e-06)
tensor(4.8519e-06)
tensor(5.0552e-06)
Testing:  3 128 0.0001
tensor(6.9695e-06)
tensor(7.5448e-06)
tensor(7.4468e-06)
tensor(5.7960e-06)
tensor(6.6154e-06)
Testing:  3 256 0.0002
tensor(5.4048e-06)
tensor(4.9082e-06)
tensor(5.6083e-06)
tensor(4.9279e-06)
tensor(4.8404e-06)
Testing:  3 256 0.001
tensor(0.0001)
tensor(0.0001)
tensor(0.0001)
tensor(0.0001)
tensor(0.0001)
Testing:  3 256 0.0001
tensor(5.1952e-06)
tensor(2.5160e-06)
tensor(5.9744e-06)
tensor(5.4704e-06)
tensor(5.2759e-06)
Testing:  3 512 0.0002
tensor(6.9697e-06)
tensor(6.4950e-06)
tensor(4.4270e-06)
tensor(5.2333e-06)
tensor(4.4260e-06)
Testing:  3 512 0.001
tensor(2.7812)
tensor(0.4395)
tensor(1.1945)
tensor(0.1425)
tensor(395.5543)
Testing:  3 512 0.0001
tensor(4.7010e-06)
tensor(2.2929e-06)
tensor(2.2241e-06)
tensor(2.8280e-06)
tensor(4.7464e-06)
Te

In [ ]:
loss_avg = torch.zeros(5)
loss_std = torch.zeros(5)
for t in range(T):
  model = torch.load("drive/MyDrive/MeasureMaps/Transformer/trial-"+str(t)+"-depth-"+str(best_d)+"-width-"+str(best_w)+"-lr-"+str(best_lr)+"-epoch-1000.pt")
  loss = test_transformer_fnn_cs(model, test_loader)
  loss_avg[t] = loss.mean()
  loss_std[t] = loss.std()
print(loss_avg.mean(), loss_avg.std())

<ipython-input-40-cb5840bb86f8>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("drive/MyDrive/MeasureMaps/Transformer/trial-"+str(t)+"-depth-"+str(best_

tensor(2.8703e-06) tensor(1.2511e-06)


In [ ]:
def test_GNN(model, loader):
  N = len(loader)
  loss = torch.zeros(N)
  model.eval()
  i = 0
  for data in loader:
    data = data.to('cuda')
    Y_pred = model(data.x, data.edge_index, data.batch)
    loss[i] = torch.nn.functional.mse_loss(Y_pred, data.y).cpu().detach()
    i += 1
  return loss

In [ ]:
depths = [3,4,5]
widths = [128,256,512]
lrs = [2e-4, 1e-3, 1e-4]
T = 5

best_loss = 100000
best_d = -1
best_w = -1
best_lr = -1

for d in depths: # depth
  for h in widths: # width
    for lr in lrs:
      avg_loss = 0
      print("Testing: ",d,h,lr)
      for t in range(T):
        model = torch.load("drive/MyDrive/MeasureMaps/GNN/GraphConv KNN/trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-1000.pt")
        loss = test_GNN(model, val_loader_gnn_knn)
        avg_loss += loss.mean()/T
      print(avg_loss)
      if avg_loss < best_loss:
        best_loss = avg_loss
        best_d = d
        best_w = h
        best_lr = lr
print(best_loss, best_d, best_w, best_lr)

Testing:  3 128 0.0002


<ipython-input-49-59282bab970b>:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("drive/MyDrive/MeasureMaps/GNN/GraphConv KNN/trial-"+str(t)+"-depth-"+st

tensor(0.0005)
Testing:  3 128 0.001
tensor(0.0003)
Testing:  3 128 0.0001
tensor(0.0008)
Testing:  3 256 0.0002
tensor(0.0004)
Testing:  3 256 0.001
tensor(0.0003)
Testing:  3 256 0.0001
tensor(0.0006)
Testing:  3 512 0.0002
tensor(0.0002)
Testing:  3 512 0.001
tensor(0.0003)
Testing:  3 512 0.0001
tensor(0.0003)
Testing:  4 128 0.0002
tensor(0.0007)
Testing:  4 128 0.001
tensor(0.0003)
Testing:  4 128 0.0001
tensor(0.0014)
Testing:  4 256 0.0002
tensor(0.0003)
Testing:  4 256 0.001
tensor(0.0003)
Testing:  4 256 0.0001
tensor(0.0007)
Testing:  4 512 0.0002
tensor(0.0003)
Testing:  4 512 0.001
tensor(0.0003)
Testing:  4 512 0.0001
tensor(0.0003)
Testing:  5 128 0.0002
tensor(0.0007)
Testing:  5 128 0.001
tensor(0.0004)
Testing:  5 128 0.0001
tensor(0.0023)
Testing:  5 256 0.0002
tensor(0.0004)
Testing:  5 256 0.001
tensor(0.0003)
Testing:  5 256 0.0001
tensor(0.0008)
Testing:  5 512 0.0002
tensor(0.0004)
Testing:  5 512 0.001
tensor(0.0003)
Testing:  5 512 0.0001
tensor(0.0005)
tensor

In [ ]:
loss_avg = torch.zeros(5)
loss_std = torch.zeros(5)
for t in range(T):
  model = torch.load("drive/MyDrive/MeasureMaps/GNN/GraphConv KNN/trial-"+str(t)+"-depth-"+str(best_d)+"-width-"+str(best_w)+"-lr-"+str(best_lr)+"-epoch-1000.pt")
  loss = test_GNN(model, test_loader_gnn_knn)
  loss_avg[t] = loss.mean()
  loss_std[t] = loss.std()
print(loss_avg.mean(), loss_avg.std())

<ipython-input-50-78f0ad510560>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("drive/MyDrive/MeasureMaps/GNN/GraphConv KNN/trial-"+str(t)+"-depth-"+str

tensor(0.0002) tensor(1.2755e-05)


In [ ]:
depths = [3,4,5]
widths = [128,256,512]
lrs = [2e-4, 1e-3, 1e-4]
T = 5

best_loss = 100000
best_d = -1
best_w = -1
best_lr = -1

for d in depths: # depth
  for h in widths: # width
    for lr in lrs:
      avg_loss = 0
      print("Testing: ",d,h,lr)
      for t in range(T):
        model = torch.load("drive/MyDrive/MeasureMaps/GNN/GraphConv Full/trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-1000.pt")
        loss = test_GNN(model, val_loader_gnn_knn)
        avg_loss += loss.mean()/T
      print(avg_loss)
      if avg_loss < best_loss:
        best_loss = avg_loss
        best_d = d
        best_w = h
        best_lr = lr
print(best_loss, best_d, best_w, best_lr)

Testing:  3 128 0.0002


<ipython-input-51-b9836fcafe50>:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("drive/MyDrive/MeasureMaps/GNN/GraphConv Full/trial-"+str(t)+"-depth-"+s

tensor(17.3385)
Testing:  3 128 0.001
tensor(0.5045)
Testing:  3 128 0.0001
tensor(70.8661)
Testing:  3 256 0.0002
tensor(7.8817)
Testing:  3 256 0.001
tensor(0.4313)
Testing:  3 256 0.0001
tensor(18.2655)
Testing:  3 512 0.0002
tensor(0.6840)
Testing:  3 512 0.001
tensor(0.1099)
Testing:  3 512 0.0001
tensor(4.4676)
Testing:  4 128 0.0002
tensor(45.6931)
Testing:  4 128 0.001
tensor(1.6845)
Testing:  4 128 0.0001
tensor(84.6195)
Testing:  4 256 0.0002
tensor(7.1467)
Testing:  4 256 0.001
tensor(0.6459)
Testing:  4 256 0.0001
tensor(15.2830)
Testing:  4 512 0.0002
tensor(2.2555)
Testing:  4 512 0.001
tensor(0.5295)
Testing:  4 512 0.0001
tensor(1.7258)
Testing:  5 128 0.0002
tensor(28.0823)
Testing:  5 128 0.001
tensor(3.5926)
Testing:  5 128 0.0001
tensor(200.1395)
Testing:  5 256 0.0002
tensor(10.9196)
Testing:  5 256 0.001
tensor(2.0824)
Testing:  5 256 0.0001
tensor(54.6190)
Testing:  5 512 0.0002
tensor(2.2296)
Testing:  5 512 0.001
tensor(0.7263)
Testing:  5 512 0.0001
tensor(12.

In [ ]:
loss_avg = torch.zeros(5)
loss_std = torch.zeros(5)
for t in range(T):
  model = torch.load("drive/MyDrive/MeasureMaps/GNN/GraphConv Full/trial-"+str(t)+"-depth-"+str(best_d)+"-width-"+str(best_w)+"-lr-"+str(best_lr)+"-epoch-1000.pt")
  loss = test_GNN(model, test_loader_gnn_knn)
  loss_avg[t] = loss.mean()
  loss_std[t] = loss.std()
print(loss_avg.mean(), loss_avg.std())

<ipython-input-52-431a55c39420>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("drive/MyDrive/MeasureMaps/GNN/GraphConv Full/trial-"+str(t)+"-depth-"+st

tensor(0.1157) tensor(0.0496)


In [ ]:
depths = [3,4,5]
widths = [128,256,512]
lrs = [2e-4, 1e-3, 1e-4]
T = 5

best_loss = 100000
best_d = -1
best_w = -1
best_lr = -1

for d in depths: # depth
  for h in widths: # width
    for lr in lrs:
      avg_loss = 0
      print("Testing: ",d,h,lr)
      for t in range(T):
        model = torch.load("drive/MyDrive/MeasureMaps/GNN/TransformerConv KNN/trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-1000.pt")
        loss = test_GNN(model, val_loader_gnn_knn)
        avg_loss += loss.mean()/T
      print(avg_loss)
      if avg_loss < best_loss:
        best_loss = avg_loss
        best_d = d
        best_w = h
        best_lr = lr
print(best_loss, best_d, best_w, best_lr)

Testing:  3 128 0.0002


<ipython-input-53-bdb98e46c4f1>:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("drive/MyDrive/MeasureMaps/GNN/TransformerConv KNN/trial-"+str(t)+"-dept

tensor(0.0002)
Testing:  3 128 0.001
tensor(0.0002)
Testing:  3 128 0.0001
tensor(0.0002)
Testing:  3 256 0.0002
tensor(0.0002)
Testing:  3 256 0.001
tensor(0.0001)
Testing:  3 256 0.0001
tensor(0.0002)
Testing:  3 512 0.0002
tensor(0.0001)
Testing:  3 512 0.001
tensor(9.8128e-05)
Testing:  3 512 0.0001
tensor(0.0001)
Testing:  4 128 0.0002
tensor(0.0001)
Testing:  4 128 0.001
tensor(9.1806e-05)
Testing:  4 128 0.0001
tensor(0.0002)
Testing:  4 256 0.0002
tensor(8.8895e-05)
Testing:  4 256 0.001
tensor(5.4922e-05)
Testing:  4 256 0.0001
tensor(0.0001)
Testing:  4 512 0.0002
tensor(5.2074e-05)
Testing:  4 512 0.001
tensor(5.0404e-05)
Testing:  4 512 0.0001
tensor(6.7150e-05)
Testing:  5 128 0.0002
tensor(0.0001)
Testing:  5 128 0.001
tensor(5.7378e-05)
Testing:  5 128 0.0001
tensor(0.0001)
Testing:  5 256 0.0002
tensor(5.7305e-05)
Testing:  5 256 0.001
tensor(3.2874e-05)
Testing:  5 256 0.0001
tensor(8.0008e-05)
Testing:  5 512 0.0002
tensor(3.2753e-05)
Testing:  5 512 0.001
tensor(2.81

In [ ]:
loss_avg = torch.zeros(5)
loss_std = torch.zeros(5)
for t in range(T):
  model = torch.load("drive/MyDrive/MeasureMaps/GNN/TransformerConv KNN/trial-"+str(t)+"-depth-"+str(best_d)+"-width-"+str(best_w)+"-lr-"+str(best_lr)+"-epoch-1000.pt")
  loss = test_GNN(model, test_loader_gnn_knn)
  loss_avg[t] = loss.mean()
  loss_std[t] = loss.std()
print(loss_avg.mean(), loss_avg.std())

<ipython-input-54-a1fa37ae5236>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("drive/MyDrive/MeasureMaps/GNN/TransformerConv KNN/trial-"+str(t)+"-depth

tensor(2.7917e-05) tensor(2.7811e-06)


In [ ]:
depths = [3,4,5]
widths = [128,256,512]
lrs = [2e-4, 1e-3, 1e-4]
T = 5

best_loss = 100000
best_d = -1
best_w = -1
best_lr = -1

for d in depths: # depth
  for h in widths: # width
    for lr in lrs:
      avg_loss = 0
      print("Testing: ",d,h,lr)
      for t in range(T):
        model = torch.load("drive/MyDrive/MeasureMaps/GNN/TransformerConv Full/trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-1000.pt")
        loss = test_GNN(model, val_loader_gnn_knn)
        avg_loss += loss.mean()/T
      print(avg_loss)
      if avg_loss < best_loss:
        best_loss = avg_loss
        best_d = d
        best_w = h
        best_lr = lr
print(best_loss, best_d, best_w, best_lr)

Testing:  3 128 0.0002


<ipython-input-55-950ac492515f>:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("drive/MyDrive/MeasureMaps/GNN/TransformerConv Full/trial-"+str(t)+"-dep

tensor(0.0029)
Testing:  3 128 0.001
tensor(0.0033)
Testing:  3 128 0.0001
tensor(0.0028)
Testing:  3 256 0.0002
tensor(0.0028)
Testing:  3 256 0.001
tensor(0.0032)
Testing:  3 256 0.0001
tensor(0.0028)
Testing:  3 512 0.0002
tensor(0.0029)
Testing:  3 512 0.001
tensor(0.0032)
Testing:  3 512 0.0001
tensor(0.0027)
Testing:  4 128 0.0002
tensor(0.0032)
Testing:  4 128 0.001
tensor(0.0036)
Testing:  4 128 0.0001
tensor(0.0031)
Testing:  4 256 0.0002
tensor(0.0031)
Testing:  4 256 0.001
tensor(0.0036)
Testing:  4 256 0.0001
tensor(0.0029)
Testing:  4 512 0.0002
tensor(0.0033)
Testing:  4 512 0.001
tensor(0.0036)
Testing:  4 512 0.0001
tensor(0.0029)
Testing:  5 128 0.0002
tensor(0.0033)
Testing:  5 128 0.001
tensor(0.0035)
Testing:  5 128 0.0001
tensor(0.0031)
Testing:  5 256 0.0002
tensor(0.0033)
Testing:  5 256 0.001
tensor(0.0036)
Testing:  5 256 0.0001
tensor(0.0031)
Testing:  5 512 0.0002
tensor(0.0033)
Testing:  5 512 0.001
tensor(0.0033)
Testing:  5 512 0.0001
tensor(0.0032)
tensor

In [ ]:
loss_avg = torch.zeros(5)
loss_std = torch.zeros(5)
for t in range(T):
  model = torch.load("drive/MyDrive/MeasureMaps/GNN/TransformerConv Full/trial-"+str(t)+"-depth-"+str(best_d)+"-width-"+str(best_w)+"-lr-"+str(best_lr)+"-epoch-1000.pt")
  loss = test_GNN(model, test_loader_gnn_knn)
  loss_avg[t] = loss.mean()
  loss_std[t] = loss.std()
print(loss_avg.mean(), loss_avg.std())

<ipython-input-56-9dc67e484d99>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("drive/MyDrive/MeasureMaps/GNN/TransformerConv Full/trial-"+str(t)+"-dept

tensor(0.0025) tensor(0.0001)
